In [ ]:
#import os
#import sys

#current_dir =  os.getcwd() 

#project_root = os.path.abspath(os.path.join(current_dir, '..','..','..'))
#sys.path.append(project_root)
#above setup not requried as it is referencing the package from the wheel file
 

: 

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")
#catalog = "citibike_dev"

In [ ]:
#from src.citibike.citibike_utils import get_trip_duration_mins
#from src.utils.datetime_utils import timestamp_to_date_col
#src is not required as the package is referenced from the wheel file

from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
df =spark.read.table(f"{catalog}.bronze.jc_citibike")



In [ ]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [ ]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [ ]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.silver.jc_citibike")